In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

In [14]:
# Importing a dataset
df =  pd.read_excel('data/GymSubs.xlsx')

In [15]:
df.head()

,user_id,target,name,sex,dob,location,location_population,occupation,hobbies,daily_commute,friends_number,relationship_status,education,credit_card_type
0,0,1,Halina,NaN,1982-08-07,Piastów,22732,Teaching professionals,Fitness,46.0,196,Single,NaN,Visa
1,1,0,Eustachy,male,1971-10-28,Sokółka,18331,General and keyboard clerks,"LARPing,Foreign language learning,Netball",55.0,243,Single,NaN,NaN
2,2,1,Egon,NaN,2000-07-05,Łaskarzew,4879,Protective services workers,"Bodybuilding,Kabaddi",90.0,191,In relationship,3.0,NaN
3,3,0,Eulalia,female,1992-06-10,Bydgoszcz,352313,Customer services clerks,Badminton,88.0,164,In relationship,3.0,Visa
4,4,0,Hilary,NaN,1975-01-09,Osieczna,2322,Refuse workers and other elementary workers,"Fitness,Embroidery,Lacemaking",40.0,119,Married with kids,5.0,NaN


In [16]:
NA = pd.DataFrame(data=[df.notna().sum().tolist(),
            df.isna().sum().tolist(), ["{:.2f}".format(i)+'%' \
           for i in (df.isna().sum()/df.shape[0]*100).tolist()]], 
           columns=df.columns, index=['Not NA Count', 'NA Count', 'NA Percent']).transpose()
NA

,Not NA Count,NA Count,NA Percent
user_id,4000,0,0.00%
target,4000,0,0.00%
name,3982,18,0.45%
sex,3616,384,9.60%
dob,3606,394,9.85%
location,4000,0,0.00%
location_population,4000,0,0.00%
occupation,4000,0,0.00%
hobbies,3320,680,17.00%
daily_commute,3595,405,10.12%


In [19]:
hobby_columns = df['hobbies'].str.get_dummies(',') # Save hobbies column as one hot encorder

# Łączenie nowych kolumn z oryginalnym dataframe
df_encoded = pd.concat([df, hobby_columns], axis=1)

# Wyświetlanie zdekodowanego dataframe
df_encoded.head()

,user_id,target,name,sex,dob,location,location_population,occupation,hobbies,daily_commute,...,Wood carving,Woodworking,Worldbuilding,Writing,Yo-yoing,Yoga,amateur radio,role-playing games,scrapbook,tabletop games
0,0,1,Halina,NaN,1982-08-07,Piastów,22732,Teaching professionals,Fitness,46.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,Eustachy,male,1971-10-28,Sokółka,18331,General and keyboard clerks,"LARPing,Foreign language learning,Netball",55.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,Egon,NaN,2000-07-05,Łaskarzew,4879,Protective services workers,"Bodybuilding,Kabaddi",90.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,Eulalia,female,1992-06-10,Bydgoszcz,352313,Customer services clerks,Badminton,88.0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,Hilary,NaN,1975-01-09,Osieczna,2322,Refuse workers and other elementary workers,"Fitness,Embroidery,Lacemaking",40.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
occupation_columns = df_encoded['occupation'].str.get_dummies(',') # Save hobbies column as one hot encorder

# Łączenie nowych kolumn z oryginalnym dataframe
df_encoded = pd.concat([df_encoded, occupation_columns], axis=1)

In [ ]:
sex_columns = df_encoded['sex'].str.get_dummies(',') # Save hobbies column as one hot encorder

# Łączenie nowych kolumn z oryginalnym dataframe
df_encoded = pd.concat([df_encoded, sex_columns], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              4000 non-null   int64         
 1   target               4000 non-null   int64         
 2   name                 3982 non-null   object        
 3   sex                  3616 non-null   object        
 4   dob                  3606 non-null   datetime64[ns]
 5   location             4000 non-null   object        
 6   location_population  4000 non-null   int64         
 7   occupation           4000 non-null   object        
 8   hobbies              3320 non-null   object        
 9   daily_commute        3595 non-null   float64       
 10  friends_number       4000 non-null   int64         
 11  relationship_status  3607 non-null   object        
 12  education            3592 non-null   float64       
 13  credit_card_type     3572 non-nul

In [ ]:
# Podział DataFrame na zbiór treningowy i testowy
df_train = df.dropna()
df_test = df[df.isnull().any(axis=1)]

# Przygotowanie danych treningowych i testowych
X_train = df_train.dropna()
y_train = df_train['sex']
X_test = df_test.dropna()

# Tworzenie modelu RandomForestClassifier
model = RandomForestClassifier()

# Trenowanie modelu na danych treningowych
model.fit(X_train, y_train)

# Uzupełnienie brakujących danych w kolumnie
imputed_values = model.predict(X_test)
df.loc[df.isnull().any(axis=1), 'sex'] = imputed_values

# Wyświetlenie zaktualizowanego DataFrame
print(df)

ValueError: could not convert string to float: 'Eulalia'